# Check `dev` stack

## `decktape`

In [1]:
! decktape automatic --chrome-arg=--no-sandbox -s 1280x960 \
    http://darribas.org/gds_course/content/slides/block_D_ii.html \
    slides.pdf

Loading page http://darribas.org/gds_course/content/slides/block_D_ii.html ...
Loading page finished with status: 200
Reveal JS plugin activated
Printing slide #/section-7 (8/8) ...              
Printed 8 slides


In [2]:
! rm slides.pdf

## `tippecanoe`

In [5]:
! wget http://darribas.org/gds4ae/_downloads/44b4bc22c042386c2c0f8dc6685ef17c/neighbourhoods.geojson
! tippecanoe -o file.mbtiles neighbourhoods.geojson

! rm file.mbtiles neighbourhoods.geojson

--2022-05-23 17:11:57--  http://darribas.org/gds4ae/_downloads/44b4bc22c042386c2c0f8dc6685ef17c/neighbourhoods.geojson
Resolving darribas.org (darribas.org)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to darribas.org (darribas.org)|185.199.109.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 383969 (375K) [application/geo+json]
Saving to: ‘neighbourhoods.geojson.1’

neighbourhoods.geoj 100%[===================>] 374.97K  --.-KB/s    in 0.03s   

2022-05-23 17:11:57 (13.0 MB/s) - ‘neighbourhoods.geojson.1’ saved [383969/383969]

tippecanoe: Tileset "file.mbtiles" already exists. You can use --force if you want to delete the old tileset.
tippecanoe: table metadata already exists


## `texbuild`

In [6]:
! wget https://github.com/darribas/constitution/raw/gh-pages/_includes/constitution.md \
    -O text.md
! pandoc -s text.md -o text.tex
! texbuild text.tex

--2022-05-23 17:12:03--  https://github.com/darribas/constitution/raw/gh-pages/_includes/constitution.md
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/darribas/constitution/gh-pages/_includes/constitution.md [following]
--2022-05-23 17:12:03--  https://raw.githubusercontent.com/darribas/constitution/gh-pages/_includes/constitution.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3269 (3.2K) [text/plain]
Saving to: ‘text.md’

text.md             100%[===================>]   3.19K  --.-KB/s    in 0s      

2022-05-23 17:12:04 (19.4 MB/s) - ‘text.md’ saved [3269/3269]

Going on...
python2 /bin/

In [7]:
! rm text.md text.pdf text.tex

## `jupyterbook` PDF builder

In [8]:
! jb create test_book
! jb build test_book --builder pdfhtml



Your book template can be found at

    test_book/


Running Jupyter-Book v0.12.3
Source Folder: /home/jovyan/work/test_book
Config Path: /home/jovyan/work/test_book/_config.yml
Output Path: /home/jovyan/work/test_book/_build/html
Running Sphinx v4.5.0
making output directory... done
[etoc] Changing master_doc to 'intro'
checking bibtex cache... out of date
parsing bibtex file /home/jovyan/work/test_book/references.bib... parsed 5 entries
myst v0.15.2: MdParserConfig(renderer='sphinx', commonmark_only=False, enable_extensions=['colon_fence', 'dollarmath', 'linkify', 'substitution', 'tasklist'], dmath_allow_labels=True, dmath_allow_space=True, dmath_allow_digits=True, dmath_double_inline=False, update_mathjax=True, mathjax_classes='tex2jax_process|mathjax_process|math|output_area', disable_syntax=[], url_schemes=['mailto', 'http', 'https'], heading_anchors=None, heading_slug_func=None, html_meta=[], footnote_transition=True, substitutions=[], sub_delimiters=['{', '}'], words_per_minut

In [9]:
! rm -rf test_book/

## `rpy2`

Example taken from [here](https://rpy2.github.io/rpy2-arrow/version/main/html/conversion.html#faster-pandas-r-conversions).

In [10]:
%load_ext rpy2.ipython

import pandas as pd

# Number or rows in the DataFrame.

_N = 500

pd_dataf = pd.DataFrame(
    {'x': range(_N), 'y': ['abc', 'def'] * (_N//2)}
)

In [11]:
%%R -i pd_dataf

rm(pd_dataf)

In [12]:
import pyarrow
from rpy2.robjects.packages import importr
import rpy2.robjects.conversion
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra

base = importr('base')

# We use the converter included in rpy2-arrow as template.
conv = rpy2.robjects.conversion.Converter(
    'Pandas to data.frame',
    template=pyra.converter
)

@conv.py2rpy.register(pd.DataFrame)
def py2rpy_pandas(dataf):
    pa_tbl = pyarrow.Table.from_pandas(dataf)
    # pa_tbl is a pyarrow table, and this is something
    # that the converter shipping with rpy2-arrow knows
    # how to handle.
    return base.as_data_frame(pa_tbl)

# We build a custom converter that is the default converter
# for ipython/jupyter shipping with rpy2, to which we add
# rules for Arrow + pandas we just made.
conv = rpy2.ipython.rmagic.converter + conv


# Test content

## `gds4ae`

In [13]:
! wget https://github.com/darribas/gds4ae/archive/refs/heads/master.zip
! unzip master.zip
! cd gds4ae-master && \
    rm -rf tests && \
	mkdir tests && \
	jupyter nbconvert --to notebook \
                      --execute \
                      --output-dir=tests \
                      --ExecutePreprocessor.timeout=600 \
                      --ExecutePreprocessor.ipython_hist_file='' \
                      content/notebooks/*.ipynb

--2022-05-23 17:14:28--  https://github.com/darribas/gds4ae/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/darribas/gds4ae/zip/refs/heads/master [following]
--2022-05-23 17:14:29--  https://codeload.github.com/darribas/gds4ae/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [          <=>       ] 198.83M  12.3MB/s    in 17s     

2022-05-23 17:14:47 (11.8 MB/s) - ‘master.zip’ saved [208485627]

Archive:  master.zip
8dcc49d7f57f3e46f448340d6190ac83526aa64c
   creating: gds4ae-master/
   creating: gds4ae-master/.github/
   creating: gds4ae-ma

In [ ]:
! rm -rf gds4ae-master master.zip

## `gds_course`

In [ ]:
! wget https://github.com/darribas/gds_course/archive/refs/heads/master.zip
! unzip master.zip
! cd gds_course-master && \
    make test

In [ ]:
! rm -rf gds_course-master master.zip

## `smds`

In [7]:
! wget https://github.com/GDSL-UL/san/archive/refs/heads/master.zip
! unzip master.zip
! cd san-master && \
    Rscript -e "library(bookdown); bookdown::render_book('index.Rmd', 'bookdown::gitbook')"

In [ ]:
! rm -rf master.zip san-master